# Integrate MERFISH objects with the Multiome dataset

In [1]:
import scanpy as sc
import os
import glob
from src.multiome_merfish_merge import *

The following code will ask for the location of the multiome data. 

For every h5ad cell-by-gene matrix that has been created and stored in a subdirectory of the folder "outputs", all categorical and continuous observations will be transferred from the multiome data to the MERFISH Anndata. 

In [19]:
adata = probe_r_or_scanpy()
list_of_scanpy_object_files = glob.glob('outputs/*/*.h5ad')

for lfile in list_of_scanpy_object_files:
    
    experiment_name = lfile.split(os.path.sep)[-2]
    a1 = sc.read(lfile)
    gene_to_id_table = pd.read_csv('DataPathogenPanel1/geneID_to_geneName_MERSCOPE_panel1.txt', sep='\t', index_col=0)

    a1, gene_name_list = modify_adata_before_anchors(experiment_name, a1, gene_to_id_table, lfile)
    
    intersection_list, adata, a1, adata_subset = create_highly_variable(a1, adata)
    
    a1.var.index = a1.var.gene_name    
    
    adata_subset, total_subscan = prepare_integration_multiome_object(adata_subset, a1)

    adata, total_subscan, a1, adata_subset = update_integration_multiome_object(adata, total_subscan, a1, adata_subset)
    
    alist = [total_subscan, a1]
    
    try_avr, a1 = find_anchors_and_transfer_labels(alist)
    
    a1.write(os.path.dirname(lfile)+os.path.sep+'cellxgene_no_motif'+experiment_name+'.h5ad')
    
    try_avr = harmony_on_merfish(try_avr)
    
    try_avr = plot_umap_merfish_rna(try_avr, experiment_name)
    
    destination = move_figure_path(lfile, experiment_name)
    
    motif_cols, partial_ad = integrate_motifs(a1)
    
    plot_motifs(partial_ad, motif_cols, destination)
    
    partial_ad.var.index.names = [None]    
    partial_ad.var = partial_ad.var.drop([partial_ad.var.columns[0]], axis = 1)    
    partial_ad.write(os.path.dirname(lfile)+os.path.sep+'cellxgene_with_motif'+experiment_name+'.h5ad')

    plot_clusters(a1, try_avr, experiment_name, lfile, destination, save = True)